In [9]:
from astroquery.sdss import SDSS
from astropy.io import fits
import csv
from astropy.coordinates import match_coordinates_sky
import astropy.units as u
from astropy.coordinates import SkyCoord
import numpy as np
from specutils import Spectrum1D
import pandas as pd
import seaborn as sns

In [29]:
eboss_fits = fits.open('eboss_dr17_summary_edapv10_cigale.fit', memmap=True)

eboss_data = eboss_fits[1].data
eboss_fits.info()
print(eboss_fits[1].columns)

Filename: eboss_dr17_summary_edapv10_cigale.fit
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU    235   1898753R x 109C   ['J', 'J', 'J', '16A', 'D', 'E', 'D', 'D', 'E', '118A', '4A', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', '9A', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', '8E', '8E', '8E', '8E', '5E', '5E', '5E', '5E', 'E', 'E', 'A', 'A', 'A', 'I', 'I', 'I']   
ColDefs(
    name = 'PLATE'; format = 'J'
    name = 'MJD'; format = 'J'
    name = 'FIBER'; format = 'J'
    name = 'PMF_STRING'; format = '16A'
    name = 'Z'; format = 'D'
    name = 'Z_ERR'; format = 'E'
    nam

In [31]:
gals_flux = eboss_data[(eboss_data['NII_6584_FLUX_ERR']>0)]

err_range = 3

# Make sure relavent emission lines have well measured fluxes
quality_gals = gals_flux[((gals_flux['H_ALPHA_FLUX']/gals_flux['H_ALPHA_FLUX_ERR'])>err_range) &
                (((gals_flux['NII_6584_FLUX']/gals_flux['NII_6584_FLUX_ERR'])>err_range) | (gals_flux["H_GAMMA_EW"]>10)) & 
                (gals_flux['Z']<0.15)] #signal to noise ratio filter

cut_gals = quality_gals[(np.log10(quality_gals['NII_6584_FLUX']/quality_gals['H_ALPHA_FLUX'])<(-1.5))]

col1 = fits.Column(name='Column1', format='D', array=cut_gals["RA"])
col2 = fits.Column(name='Column2', format='D', array=cut_gals["DEC"])
table_data = fits.BinTableHDU.from_columns([col1, col2])

new_primary_hdu = fits.PrimaryHDU()
new_hdul = fits.HDUList([new_primary_hdu, table_data])

new_hdul.writeto('modified.fits', overwrite=True)

eboss_RA = cut_gals["RA"]
eboss_DEC = cut_gals["DEC"]

C:\Users\nihad\AppData\Local\Temp\ipykernel_19968\3833484631.py:10: RuntimeWarning: divide by zero encountered in log10
  cut_gals = quality_gals[(np.log10(quality_gals['NII_6584_FLUX']/quality_gals['H_ALPHA_FLUX'])<(-1.5))]


## FIRST CrossMatch

In [58]:
first_fits = fits.open("first_survey.fits", memmap=True)

first_data = first_fits[1].data
first_fits.info()
print(first_fits[1].columns)

first_RA = first_data["RAJ2000"]
first_DEC = first_data["DEJ2000"]
print(len(first_RA))

Filename: first_survey.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       5   ()      
  1                1 BinTableHDU     87   946432R x 16C   ['16A', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'I', 'A', 'I', 'D', 'D', 'D']   
ColDefs(
    name = 'FIRST'; format = '16A'
    name = 'RAJ2000'; format = 'D'; unit = 'deg'
    name = 'DEJ2000'; format = 'D'; unit = 'deg'
    name = 'p(S)'; format = 'D'
    name = 'Fpeak'; format = 'D'; unit = 'mJy'
    name = 'Fint'; format = 'D'; unit = 'mJy'
    name = 'Rms'; format = 'D'; unit = 'mJy'
    name = 'Maj'; format = 'D'; unit = 'arcsec'
    name = 'Min'; format = 'D'; unit = 'arcsec'
    name = 'PA'; format = 'D'; unit = 'deg'
    name = 'N1'; format = 'I'; null = -32768
    name = 'c1'; format = 'A'
    name = 'N2'; format = 'I'
    name = 'Ep'; format = 'D'; unit = 'yr'
    name = 'Ep(JD)'; format = 'D'; unit = 'd'
    name = 's_Ep(JD)'; format = 'D'; unit = 'd'
)
946432


In [60]:
eboss = SkyCoord(ra=eboss_RA*u.degree, dec=eboss_DEC*u.degree)
first = SkyCoord(ra=first_RA*u.degree, dec=first_DEC*u.degree)

max_sep = 1.0 * u.arcsec
idx, d2d, d3d = eboss.match_to_catalog_sky(first)

sep_constraint = d2d < max_sep
eboss_matches = eboss[sep_constraint]
first_matches = first[idx[sep_constraint]]

print(eboss_matches)
print(first_matches)

<SkyCoord (ICRS): (ra, dec) in deg
    [( 28.29982891,  1.07781419), (214.85086864, 28.10771986)]>
<SkyCoord (ICRS): (ra, dec) in deg
    [( 28.29985833,  1.07783056), (214.8506375 , 28.10761667)]>


## VLASS Crossmatch

In [39]:
import csv

VLASS_RA = []
VLASS_DEC = []
with open('VLASS.csv', 'r') as csvfile:
    csv_reader = csv.reader(csvfile)
    for row in csv_reader:
        if row:  # Ensure the row is not empty
            VLASS_RA.append(row[3])
            VLASS_DEC.append(row[4])

In [40]:
VLASS_RA = VLASS_RA[1:]
VLASS_DEC = VLASS_DEC[1:]

In [41]:
VLASS_RA = np.array(list(map(float, VLASS_RA)))
VLASS_DEC = np.array(list(map(float, VLASS_DEC)))

VLASS = SkyCoord(ra=VLASS_RA*u.degree, dec=VLASS_DEC*u.degree)

max_sep = 2.0 * u.arcsec
idx, d2d, d3d = eboss.match_to_catalog_sky(VLASS)

sep_constraint = d2d < max_sep
eboss_matches = eboss[sep_constraint]
VLASS_matches = VLASS[idx[sep_constraint]]

print(eboss_matches)
print(VLASS_matches)

<SkyCoord (ICRS): (ra, dec) in deg
    []>
<SkyCoord (ICRS): (ra, dec) in deg
    []>


## LoTSS Crossmatch

In [25]:
LoTSS_fits = fits.open("LoTSS.fits", memmap=True)

LoTSS_data = LoTSS_fits[1].data
LoTSS_fits.info()
print(LoTSS_fits[1].columns)

LoTSS_RA = LoTSS_data["RA"]
LoTSS_DEC = LoTSS_data["DEC"]
print(len(LoTSS_RA))

Filename: LoTSS.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       4   ()      
  1                1 BinTableHDU     81   4396228R x 26C   ['24A', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', 'D', '1A', '11A', 'J', 'D']   
ColDefs(
    name = 'Source_Name'; format = '24A'
    name = 'RA'; format = 'D'; unit = 'deg'
    name = 'E_RA'; format = 'D'; unit = 'arcsec'
    name = 'DEC'; format = 'D'; unit = 'deg'
    name = 'E_DEC'; format = 'D'; unit = 'arcsec'
    name = 'Peak_flux'; format = 'D'; unit = 'beam-1 mJy'
    name = 'E_Peak_flux'; format = 'D'; unit = 'beam-1 mJy'
    name = 'Total_flux'; format = 'D'; unit = 'mJy'
    name = 'E_Total_flux'; format = 'D'; unit = 'mJy'
    name = 'Maj'; format = 'D'; unit = 'arcsec'
    name = 'E_Maj'; format = 'D'; unit = 'arcsec'
    name = 'Min'; format = 'D'; unit = 'arcsec'
    name = 'E_Min'; format = 'D'; unit = 'arcsec'
    name =

In [37]:
LoTSS = SkyCoord(ra=LoTSS_RA*u.degree, dec=LoTSS_DEC*u.degree)

max_sep = 2.0 * u.arcsec
idx, d2d, d3d = eboss.match_to_catalog_sky(LoTSS)

sep_constraint = d2d < max_sep
eboss_matches = eboss[sep_constraint]
LoTSS_matches = LoTSS[idx[sep_constraint]]

print(eboss_matches)
print(LoTSS_matches)

<SkyCoord (ICRS): (ra, dec) in deg
    [(183.30780513, 39.94642162), (  4.48175936, 18.87210016),
     (164.51733577, 51.81236532), (231.3329221 , 56.9426449 ),
     (163.47883093, 57.12761991), (163.3137729 , 31.8581494 )]>
<SkyCoord (ICRS): (ra, dec) in deg
    [(183.30782061, 39.94673049), (  4.481618  , 18.8719536 ),
     (164.5167559 , 51.81263417), (231.33274714, 56.94283192),
     (163.47934877, 57.12742648), (163.31328814, 31.85844893)]>


## eROSITA crossmatch

In [52]:
eROSITA_fits = fits.open("eROSITA.fits", memmap=True)

eROSITA_data = eROSITA_fits[1].data
eROSITA_fits.info()
print(eROSITA_fits[1].columns)

eROSITA_RA = eROSITA_data["ERO_RA_CORR"]
eROSITA_DEC = eROSITA_data["ERO_DEC_CORR"]
print(len(eROSITA_RA))

Filename: eROSITA.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      16   (121244,)   uint8   
  1  Joined        1 BinTableHDU    997   27369R x 241C   ['22A', 'J', 'D', 'D', 'E', 'E', 'E', 'E', '11A', 'D', 'D', 'E', 'I', 'J', 'D', 'D', 'J', 'E', 'J', '8A', 'L', '12A', 'K', 'K', 'J', 'J', 'E', 'E', 'E', 'E', 'D', 'D', 'E', 'E', 'L', 'J', '32A', 'E', '8A', 'J', '16A', 'L', '8A', 'D', 'D', 'D', 'L', 'L', 'L', 'L', 'L', 'L', '11A', 'D', 'D', 'E', 'E', 'E', 'E', 'E', 'E', 'K', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'J', 'L', '4A', 'I', 'I', '20A', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'E', 'I', 'I', 'E', 'E', 

In [56]:
eROSITA = SkyCoord(ra=eROSITA_RA*u.degree, dec=eROSITA_DEC*u.degree)

max_sep = 2.0 * u.arcsec
idx, d2d, d3d = eboss.match_to_catalog_sky(eROSITA)

sep_constraint = d2d < max_sep
eboss_matches = eboss[sep_constraint]
eROSITA_matches = eROSITA[idx[sep_constraint]]

print(eboss_matches)
print(eROSITA_matches)

<SkyCoord (ICRS): (ra, dec) in deg
    []>
<SkyCoord (ICRS): (ra, dec) in deg
    []>


Plot the spectra for all the matches
Figure out the broad emission line (hard)